#데이터마님 실기 1회

In [ ]:
#작업1유형


import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')


#1) 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0~9 : 0 , 10~19 : 10)
df['나이대'] = df.age // 10 * 10
print(df.나이대.value_counts().index[0])


#2) 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?
df['나이대'] = df.age // 10 * 10
print(df.나이대.value_counts().values[0])


#3) 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?
print(len(df.loc[(df.age>=25) & (df.age<29) & (df.housing == 'yes')]))


#4) numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?
cols = []
for i in df.select_dtypes(include=object):
    cols.append([i, df[i].nunique()])
    
print(pd.DataFrame(cols).sort_values(1, ascending=False).values[0][0])


#5) balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?
print(df.loc[df.balance >= df.balance.mean()].sort_values('ID', ascending=False).head(100).balance.mean())


#6) 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)
print(df[['day', 'month']].value_counts().index[0])


#7) 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라
from scipy.stats import shapiro
result = shapiro(df[df.job =='unknown'].age)[1]
print(result)


#8) age와 balance의 상관계수를 구하여라
print(df[['age', 'balance']].corr().iloc[0,1])


#9) y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라
v = pd.crosstab(df.y,df.education)
from scipy.stats import chi2_contingency
chi2 , p ,dof, expected = chi2_contingency(v)
display(v)
print(p)


#10) 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?
dm = df.groupby(['job', 'marital']).size().reset_index()
pdm = dm.pivot_table(index='job', columns='marital')[0]
pdm = pdm.fillna(0)
pdm['ratio'] = pdm['divorced']/pdm['married']
print(pdm.sort_values('ratio').ratio.values[-1])

30
5056
504
job
3473.73
(15, 'may')
0.1961131989955902
0.10198734763981462


education,primary,secondary,tertiary,unknown
y,,,,
no,1424,4555,2559,365
yes,456,1813,1516,182


7.901201277473551e-29
0.2831050228310502


In [ ]:
#작업2유형
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')


#1. 결측치 확인
#print(train.head())
#print(test.head())
#print(submission.head())

#print(train.shape)
#print(test.shape)

#print(train.isnull().sum())
#print(test.isnull().sum())

#print(train.info())
#print(test.info())

#job, education, #contact, poutcome    
#print(train[['job', 'education', 'contact', 'poutcome']].head(20))

#for i in train.select_dtypes(include=object):
#    print(train[i].value_counts())

#print(train.describe())


#2. x,y
#'ID', 'age', 'job', 'marital', 'education', 'default', 'balance',
#       'housing', 'loan', 'contact', 'day', 'month', 'campaign', 'pdays',
#       'previous', 'poutcome', 'y'
x_train_id = train['ID']
x_test_id = test['ID']

x_train = train.drop(columns=['ID', 'y'], axis=1)
x_test = test.drop(columns=['ID'], axis=1)
y_train = train['y']
y_train = y_train.map(lambda x: 1 if x=='yes' else 0)

#3. 라벨인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

ob_cols = x_train.select_dtypes(include=object).columns
for i in ob_cols:
    x_train[i] = le.fit_transform(x_train[i])
    x_test[i] = le.fit_transform(x_test[i])

x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

int_cols = x_train.select_dtypes(exclude=object).columns
x_train[int_cols] = scaler.fit_transform(x_train[int_cols])
x_test[int_cols] = scaler.fit_transform(x_test[int_cols])


#5. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=2022, test_size=0.3, stratify=y_train)


#6. 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred_lr1 = lr.predict(x_val)
pred_lr2 = lr.predict_proba(x_val)[:,1]

rf = RandomForestClassifier(random_state=7778)
rf.fit(x_train, y_train)
pred_rf1 = rf.predict(x_val)
pred_rf2 = rf.predict_proba(x_val)[:,1]

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb.fit(x_train, y_train)
pred_xgb1 = xgb.predict(x_val)
pred_xgb2 = xgb.predict_proba(x_val)[:,1]

#print(x_train.head(5))
#print(x_train.shape, x_test.shape, y_train.shape) #(12870, 15) (4291, 15) (12870,)



#6. 검증
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

print('------------------------------')
print(f1_score(y_val, pred_lr1))
print(f1_score(y_val, pred_rf1))
print(f1_score(y_val, pred_xgb1))

print('------------------------------')
print(accuracy_score(y_val, pred_lr1))
print(accuracy_score(y_val, pred_rf1))
print(accuracy_score(y_val, pred_xgb1))

print('------------------------------')
print(roc_auc_score(y_val, pred_lr2))
print(roc_auc_score(y_val, pred_rf2))
print(roc_auc_score(y_val, pred_xgb2))



#7. 제출
submission['predict'] = xgb.predict_proba(x_test)[:,1]

print(submission)
submission.to_csv('000000.csv', index=False)



------------------------------
0.3541795665634675
0.553014553014553
0.552801724137931
------------------------------
0.7298627298627298
0.7772597772597772
0.785029785029785
------------------------------
0.7279252097694188
0.7809022523273631
0.7858660873559458
         ID   predict
0     53608  0.808052
1     51055  0.702534
2     52573  0.108903
3     50458  0.131618
4     52272  0.122303
...     ...       ...
4286  50332  0.117962
4287  51293  0.762411
4288  54023  0.237206
4289  53775  0.478051
4290  51945  0.139270

[4291 rows x 2 columns]


#데이터마님 실기 2회

In [ ]:
#1. 작업 1유형


import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')


#1) 성별이 Male인 환자들의 age의 평균값
df['age'] = df['age'].replace('*82','82').astype('int64')
#print(df[df.gender == 'Male'].age.mean())


#2) bmi컬럼의 결측치를 bmi컬럼의 결측치를 제외한 나머지 값들의 중앙값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지
df.bmi = df.bmi.fillna(df.bmi.median())
#print(round(df.bmi.mean(),3))


#3) bmi컬럼의 각 결측치들을 직전의 행의 bmi값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리
fi = df['bmi'].fillna(method = 'ffill')
#print(round(fi.mean(),3))


#4) bmi컬럼의 각 결측치들을 결측치를 가진 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 대체된 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
df['age'] = df['age'].replace('*82','82').astype('int64')
df['나이대'] = df.age//10*10
#결측치 가진 환자 나이대의 평균 bmi값과 결측치 index
mean = df[df.bmi.notnull()].groupby('나이대').bmi.mean()
idx =df.loc[df.bmi.isnull(),['age','bmi']].index
# 결측치 매핑
dic = { x:y for x,y in mean.items()}
df.loc[df.bmi.isnull(),'bmi']  =(df[df.bmi.isnull()].나이대).map(lambda x : dic[x])
#print(df.bmi.mean())


#5) avg_glucose_level 컬럼의 값이 200이상인 데이터를 모두 199로 변경하고 stroke값이 1인 데이터의 avg_glucose_level값의 평균을 소수점이하 3자리 까지
df.loc[(df.avg_glucose_level >= 200), df.avg_glucose_level] = 199
round(df[df.stroke == 1].avg_glucose_level.mean(), 3)

131.428

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

#6) Attack컬럼의 값을 기준으로 내림차순정렬 했을때 상위 400위까지 포켓몬들과 401~800위까지의 포켓몬들에서 전설포켓몬(Legendary컬럼)의 숫자 차이는?
up = df.sort_values('Attack', ascending=False).reset_index(drop=True)[:400].Legendary.sum()
down = df.sort_values('Attack', ascending=False).reset_index(drop=True)[400:800].Legendary.sum()
#print(up - down)


#7) Type 1 컬럼의 종류에 따른 Total 컬럼의 평균값을 내림차순 정렬했을때 상위 3번째 Type 1은 무엇인가?
df.groupby(['Type 1']).Total.mean().sort_values(ascending=False).index[2]

#8) 결측치가 존재하는 행을 모두 지운 후 처음부터 순서대로 60% 데이터를 추출하여 Defense컬럼의 1분위수
df.dropna()[:int(len(df.dropna())*0.6)].Defense.quantile(0.25)


#9) Type 1 컬럼의 속성이 Fire인 포켓몬들의 Attack의 평균이상인 Water속성의 포켓몬 수
df[df[df['Type 1'] =='Fire'].Attack.mean() <= df.Attack].loc[df['Type 1']== 'Water'].shape[0]


#10) 각 세대 중(Generation 컬럼)의 Speed와 Defense 컬럼의 차이(절댓값)이 가장 큰 세대
df_G = df.groupby(['Generation'])[['Speed', 'Defense']].mean()
df_G['new'] = abs(df_G.Speed-df_G.Defense)
df_G.sort_values('new').index[-1]

2

In [ ]:
#2유형: 뇌졸증 발생여부 예측


import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv')

#display(train.head())
#display(test.head())


#1. 결측치 확인
#print(train.shape) #(1137, 12)
#print(test.shape) #(932, 11)

#print(train.isnull().sum()) #bmi
#print(test.isnull().sum()) #bmi

#train.info()
#test.info()

#for i in test.select_dtypes(exclude=object):
#  print(test[i].value_counts())                #age-*82


#2. x,y
train['age'] = train['age'].replace('*82', '82').astype(int)

train.bmi = train.bmi.fillna(train.bmi.mean())
test.bmi = test.bmi.fillna(test.bmi.mean())

x_tr = train.drop(['id','stroke'], axis=1)
y_tr = train['stroke']
x_test_id = test['id']
x_test = test.drop(['id'], axis=1)

#print(x_tr.shape, x_test.shape)


#3. 라벨인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = x_tr.select_dtypes(include=object).columns

for i in cols:
  x_tr[i] = le.fit_transform(x_tr[i])
  x_test[i] = le.transform(x_test[i])

#print(x_tr.shape, x_test.shape)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

cols2 = ['age', 'avg_glucose_level', 'bmi']

x_tr[cols2] = scaler.fit_transform(x_tr[cols2])
x_test[cols2] = scaler.fit_transform(x_test[cols2])

#print(x_tr.shape, x_test.shape)


#5. split
from sklearn.model_selection import train_test_split
x_tr, x_tst, y_tr, y_tst = train_test_split(x_tr, y_tr, random_state=2022, test_size=0.33, stratify=y_tr)

#print(x_tr.shape, x_test.shape)


#6. 모델링 : 분류, 발생여부 예측
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

lr = LogisticRegression()
lr.fit(x_tr, y_tr)
pred1 = lr.predict(x_tst)

rf = RandomForestClassifier()
rf.fit(x_tr, y_tr)
pred2 = rf.predict(x_tst)

xg = XGBClassifier()
xg.fit(x_tr, y_tr)
pred3 = xg.predict(x_tst)


#7. 검증 : r2_score, accuracy_score
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

print('----------f1_score----------')
print(f1_score(y_tst, pred1))
print(f1_score(y_tst, pred3))

print('----------accuracy----------')
print(accuracy_score(y_tst, pred1))
print(accuracy_score(y_tst, pred3))

print('----------precision----------')
print(precision_score(y_tst, pred1))
print(precision_score(y_tst, pred3))

print('----------recall----------')
print(recall_score(y_tst, pred1))
print(recall_score(y_tst, pred3))

----------f1_score----------
0.14545454545454545
0.2388059701492537
----------accuracy----------
0.875
0.8643617021276596
----------precision----------
0.4
0.36363636363636365
----------recall----------
0.08888888888888889
0.17777777777777778


#3) 3회차


In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/spotify/spotify.csv')

#1) 데이터는 현재 년도별 100곡이 인기순으로 정렬되어 있다. 각 년도별 1~100위의 랭킹을 나타내는 rank컬럼을 만들고 매년도 1위의 bpm컬럼의 평균값
df = df.dropna()
df['rank']=list(range(1,101))*10
#print(df.loc[df['rank']==1].bpm.mean())


#2) 2015년도에 가장많은 top100곡을 올린 artist는 누구인가?
df[df['top year']==2015].groupby('artist').size().sort_values(ascending=False).index[0]
result = df[df['top year'] ==2015].artist.value_counts().index[0]


#3) 년도별 rank값이 1~10위 까지의 곡들 중 두번째로 많은 top genre는 무엇인가?
df.loc[df['rank']<=10]['top genre'].value_counts().index[1]


#4) 피처링의 경우 title에 표시된다. 피처링을 가장 많이 해준 가수는 누구인가?
df.title.str.split('feat.').str[1].dropna().str[:-1].dropna().value_counts().index[0]


#5) top year 년도를 기준으로 발매일(year released)과 top100에 진입한 일자 (top year)가 다른 곡의 숫자를 count 했을때 가장 많은 년도는?
df.loc[df['year released'] != df['top year']]['top year'].value_counts().index[0]


#6) artist 컬럼의 값에 대소문자 상관없이 q 단어가 들어가는 아티스트는 몇명인가?
df[df.artist.str.lower().str.contains('q')].artist.nunique()


#7) 년도 상관없이 전체데이터에서 1~50위와 51~100위간의 dur 컬럼의 평균값의 차이는?
df[df['rank']<=50].dur.mean() - df[df['rank']>50].dur.mean()


#8) title을 띄어쓰기 단어로 구분 했을때 가장 많이 나온 단어는 무엇인가? (대소문자 구분 x)
df.title.str.lower().str.split('\(feat.').str[0].str.split().explode().value_counts().index[0]


#9) 년도별 nrgy값의 평균값을 구할때 최대 평균값과 최소 평균값의 차이를 구하여라
data = df.groupby('top year').nrgy.mean().sort_values(ascending=False)
data.values[0]-data.values[-1]


#10) artist중 artist type 타입을 여러개 가지고 있는 artist는 누구인가
df.groupby(['artist','artist type']).size().groupby('artist').size().sort_values(ascending=False).index[0]
df[['artist','artist type']].value_counts().reset_index().artist.value_counts().index[0]

#print(df.head(10))

'Rudimental'

In [ ]:
#2유형

import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/y_train.csv")
test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_test.csv")


#display(x_train.head())
#display(y_train.head())


#1. EDA
#print(x_train.isnull().sum())
#print(y_train.isnull().sum())
#print(test.isnull().sum())

#print(x_train.info())
#print(y_train.info())
#print(test.info())


#2. x,y
x_test_id = test['ID']
x_test = test.drop(columns=['ID'])
x_train = x_train.drop(columns=['ID'])
y_train = y_train.drop(columns=['ID'])


#3. 라벨
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


#5. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=2022, stratify=y_train, test_size=0.33)

print(x_train.shape, x_test.shape)


#6. 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict(x_val)

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict(x_val)

xg = XGBClassifier()
xg.fit(x_train, y_train)
pred3 = xg.predict(x_val)


#7. 검증
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print(accuracy_score(y_val,pred1))
print(accuracy_score(y_val,pred2))
print(accuracy_score(y_val,pred3))
print('----------------------------')
print(f1_score(y_val,pred1))
print(f1_score(y_val,pred2))
print(f1_score(y_val,pred3))
print('----------------------------')
print(precision_score(y_val,pred1))
print(precision_score(y_val,pred2))
print(precision_score(y_val,pred3))
print('----------------------------')
print(recall_score(y_val,pred1))
print(recall_score(y_val,pred2))
print(recall_score(y_val,pred3))



#8. 결과
pred = xg.predict(x_test)
result = pd.DataFrame({'ID': x_test_id, 'predict':pred}).to_csv('00000.csv', index=False)

(3115, 64) (1163, 64)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, w

0.5726384364820847
0.9954397394136808
0.996742671009772
----------------------------
0.602905569007264
0.9954158480681073
0.9967384213959557
----------------------------
0.5627118644067797
1.0
0.9973890339425587
----------------------------
0.6492829204693612
0.9908735332464146
0.9960886571056062


#작업환경

In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

x_test = pd.read_csv("data/X_test.csv")
x_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

#1. 결측치
#print(x_train.shape, x_test.shape)  #(3500,10) (2482,10)
#print(x_train.isnull().sum())      #환불금액  0
#print(y_train.isnull().sum())
#print(x_test.isnull().sum())       #환불금액  0
#print(x_test.info())
#for i in y_train.select_dtypes(exclude=object):
#	print(y_train[i].value_counts())

#2. x,y
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)

x_test_id = x_test['cust_id']
x_train = x_train.drop(columns='cust_id')
x_test = x_test.drop(columns='cust_id')
y_train = y_train['gender']


#3. 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = x_train.select_dtypes(include=object).columns
cols2 = x_train.select_dtypes(exclude=object).columns

for i in cols:
	x_train[i] = le.fit_transform(x_train[i])
	x_test[i] = le.fit_transform(x_test[i])

#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train[cols2] = scaler.fit_transform(x_train[cols2])
x_test[cols2] = scaler.transform(x_test[cols2])

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=2022, test_size=0.33, stratify=y_train)

#5. 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)[:,1]

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)[:,1]

xg = XGBClassifier(eval_metric='mlogloss')
xg.fit(x_train, y_train)
pred3 = xg.predict_proba(x_val)[:,1]


from sklearn.metrics import roc_auc_score
print('로지스틱: ', roc_auc_score(y_val, pred1))
print('랜덤포레: ', roc_auc_score(y_val, pred2))
print('XGB: ', roc_auc_score(y_val, pred3))


#7. 결과
pred = rf.predict_proba(x_test)[:,1]
pd.DataFrame({'cust_id':x_test_id,'gender': pred}).to_csv('0000.csv', index=False)


# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


In [307]:
#96) 랜포: Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때, 랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")

from sklearn.ensemble import RandomForestRegressor

df_t = df.drop([df.columns[0]],axis=1)
x = df_t.drop([df.columns[-1]],axis=1)
y = df_t[df.columns[-1]]

ml = RandomForestRegressor()
ml.fit(x,y)

ml.feature_importances_


array([0.08396589, 0.03253776, 0.01352067, 0.02364019, 0.02406875,
       0.81526631, 0.00700045])